In [ ]:
#Importing Libraries

import pandas as pd
import matplotlib.pyplot as plt
import os
from itertools import combinations
from sklearn.linear_model import LinearRegression

: 

In [ ]:
#Loading CSV

file_name = "exoplanets_data.csv"
df = pd.read_csv("/Users/roshnipal/Desktop/Habitability-of-Exoplanets/exoplanets_data.csv")

In [ ]:
os.makedirs("scatter_plots", exist_ok=True)
os.makedirs("regression_plots", exist_ok=True)

In [ ]:
numeric_cols = df.select_dtypes(include=["float64", "int64"]).columns
print("Numeric columns identified for analysis:\n", numeric_cols)

In [ ]:
corr_matrix = df[numeric_cols].corr()
corr_matrix.to_csv("correlation_matrix.csv")
print("Correlation matrix saved as 'correlation_matrix.csv'")

In [ ]:
threshold = 0.5  #Correlation Threshold
strong_pairs = []

for col_x, col_y in combinations(numeric_cols, 2):
    corr = corr_matrix.loc[col_x, col_y]
    if abs(corr) >= threshold:
        strong_pairs.append((col_x, col_y, corr))

print(f"Found {len(strong_pairs)} strongly correlated pairs (|corr| >= {threshold})")

In [ ]:
for col_x, col_y, corr in strong_pairs:
    
    # Scatter plot
    
    plt.figure(figsize=(6, 4))
    plt.scatter(df[col_x], df[col_y], alpha=0.5)
    plt.xlabel(col_x)
    plt.ylabel(col_y)
    plt.title(f"Scatter: {col_y} vs {col_x} (corr={corr:.2f})")
    plt.tight_layout()
    plt.savefig(f"scatter_plots/{col_y}_vs_{col_x}.png")
    plt.close()

In [ ]:
#Linear Regression

X = df[[col_x]].dropna()
y = df[col_y].dropna()
common_index = X.index.intersection(y.index)
X = X.loc[common_index]
y = y.loc[common_index]

slope = intercept = None
if len(X) > 1:
    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    slope = model.coef_[0]
    intercept = model.intercept_

In [ ]:
#Regression Plot

plt.figure(figsize=(6, 4))
plt.scatter(X, y, alpha=0.5)
plt.plot(X, y_pred, color="red")
plt.xlabel(col_x)
plt.ylabel(col_y)
plt.title(f"Regression: {col_y} vs {col_x} (corr={corr:.2f})")
plt.tight_layout()
plt.savefig(f"regression_plots/{col_y}_vs_{col_x}_regression.png")
plt.close()

In [ ]:
# Add to summary
if 'summary_data' not in globals():
    summary_data = []
summary_data.append({
    "Variable 1": col_x,
    "Variable 2": col_y,
    "Correlation": corr,
    "Regression Slope": slope,
    "Regression Intercept": intercept
})


In [ ]:
#Save Summary

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
summary_df = pd.DataFrame(summary_data)
summary_df.to_csv("bivariate_summary.csv", index=False)
print("Summary CSV saved as 'bivariate_summary.csv'")
print("All strongly correlated plots saved in 'scatter_plots' and 'regression_plots' folders")